In [ ]:
def is_bbox_inside(inner_bbox, outer_bbox):
    """
    Check if the inner bounding box is completely inside the outer bounding box.

    Parameters:
    - inner_bbox: Tuple (x_min, y_min, x_max, y_max) representing the inner bounding box.
    - outer_bbox: Tuple (x_min, y_min, x_max, y_max) representing the outer bounding box.

    Returns:
    - True if inner_bbox is inside outer_bbox, False otherwise.
    """
    inner_x_min, inner_y_min, inner_x_max, inner_y_max = inner_bbox
    outer_x_min, outer_y_min, outer_x_max, outer_y_max = outer_bbox

    # Check if inner_bbox is inside outer_bbox
    return (
        inner_x_min >= outer_x_min and
        inner_y_min >= outer_y_min and
        inner_x_max <= outer_x_max and
        inner_y_max <= outer_y_max
    )

In [ ]:
import os

class Parameters:
    def __init__(self):
        self.base_dir = 'antrenare/'
        self.dir_pos_examples = os.path.join(self.base_dir, 'exemplePozitive')
        self.dir_neg_examples = os.path.join(self.base_dir, 'exempleNegative')
        self.dir_test_examples = ("validare/validare/")
        self.path_annotations = ("validare/validare_annotations.txt")
        self.sol_path="C:/Users/rares/Desktop/CAVA/Proiect2/rezultate/"
        
        self.dir_save_files = os.path.join(self.base_dir, 'salveazaFisiere')
        if not os.path.exists(self.dir_save_files):
            os.makedirs(self.dir_save_files)
            print('directory created: {} '.format(self.dir_save_files))
        else:
            print('directory {} exists '.format(self.dir_save_files))

        # set the parameters
        self.dim_window_height=48
        self.dim_window_width=48
        self.dim_hog_cell = 12  # dimensiunea celulei
        self.dim_descriptor_cell = 36  # dimensiunea descriptorului unei celule
        self.overlap = 0.3
        self.number_positive_examples = 6977  # numarul exemplelor pozitive
        self.number_negative_examples = 18000  # numarul exemplelor negative
        self.overlap = 0.3
        self.has_annotations = True
        self.threshold = 0


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import glob
import cv2 as cv
import pdb
import pickle
import ntpath
from copy import deepcopy
import timeit
import os
from skimage.feature import hog


class FacialDetector:
    def __init__(self, params:Parameters):
        self.params = params
        self.best_model = None

        
        
    def rescale_bbox(self,bbox, from_scale, to_scale):
        rescaled_bbox = [int(coord * to_scale / from_scale) for coord in bbox]
        return rescaled_bbox
    
    def resize_to_specific_size(self,image, target_width, target_height):
        # Resize the image to the specified size
        resized_image = cv.resize(image, (target_width, target_height))

        return resized_image
    
    
    def rescale_image(self,image, scale_factor):
        # Calculate the new dimensions after rescaling
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)

        # Resize the image
        rescaled_image = cv.resize(image, (new_width, new_height))

        return rescaled_image




    def intersection_over_union(self, bbox_a, bbox_b):
        x_a = max(bbox_a[0], bbox_b[0])
        y_a = max(bbox_a[1], bbox_b[1])
        x_b = min(bbox_a[2], bbox_b[2])
        y_b = min(bbox_a[3], bbox_b[3])

        inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

        box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
        box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

        iou = inter_area / float(box_a_area + box_b_area - inter_area)

        return iou


    def get_positive_examples(self):
        count=1
        positive_examples=[]
        people=["barney","fred","wilma","betty"]
        for person in people:
            i=1
            j=0
            folder_path = f"{self.params.base_dir}{person}"
            annotations_path=f"{self.params.base_dir}{person}_annotations.txt"
            with open(annotations_path, 'r') as file:
                lines = file.readlines()
            files = os.listdir(folder_path)
            for file in files:
                file_path = os.path.join(folder_path, file)
                img=cv.imread(file_path)
                i_str=str(i)
                while len(i_str)<4:
                    i_str="0"+i_str
                while j<len(lines) and lines[j][0:4]==i_str:
                    if lines[j].split()[5]!="":
                        poz=lines[j].split()[1:5]
                        for q in range(len(poz)):
                            poz[q]=int(poz[q])
                        xmin,ymin,xmax,ymax=poz
                        image_patch = img[(ymin):(ymax), (xmin):(xmax)]
                        image_patch=self.resize_to_specific_size(image_patch,self.params.dim_window_width,self.params.dim_window_height)
                        positive_examples.append(image_patch)
                        patch_filename = f"{count}_{lines[j].split()[-1]}.jpg" 
                        patch_filepath = os.path.join(self.params.dir_pos_examples, patch_filename)
                        cv.imwrite(patch_filepath, image_patch)
                        count+=1

                    j+=1


                i+=1
        return positive_examples

    def get_negative_examples(self):
        num_images = 4000
        negative_examples=[]
        num_negative_per_image = self.params.number_negative_examples // num_images
        people=["wilma","barney","fred","betty"]
        for person in people:
            i=1
            j=0
            folder_path = f"{self.params.base_dir}{person}"
            annotations_path=f"{self.params.base_dir}{person}_annotations.txt"
            with open(annotations_path, 'r') as file:
                lines = file.readlines()
            files = os.listdir(folder_path)
            patches=[]
            for file in files:
                c=0
                file_path = os.path.join(folder_path, file)
                img=cv.imread(file_path)
                boxes=[]
                i_str=str(i)
                while len(i_str)<4:
                    i_str="0"+i_str
                while j<len(lines) and lines[j][0:4]==i_str:
                    box=lines[j].split()[1:5]
                    for m in range(len(box)):
                        box[m]=int(box[m])
                    boxes.append(box)
                    j+=1
                i+=1

                num_rows = img.shape[0]
                num_cols = img.shape[1]
                x = np.random.randint(low=0, high=num_cols - self.params.dim_window_width, size=num_negative_per_image)
                y = np.random.randint(low=0, high=num_rows - self.params.dim_window_height, size=num_negative_per_image)
                for idx in range(len(y)):
                    c=0
                    patch =[x[idx],y[idx], x[idx]+self.params.dim_window_width,y[idx] + self.params.dim_window_height]
                    p=[]
                    nr=0
                    for box in boxes:
                        p.append(self.intersection_over_union(box,patch))
                    for v in p:
                        if v<0.05:
                            nr+=1
                    while nr!=len(boxes):
                                p=[]
                                nr=0
                                if c==50000:
                                    print(1)
                                    break
                                a = np.random.randint(low=0, high=num_cols - self.params.dim_window_width)
                                b = np.random.randint(low=0, high=num_rows - self.params.dim_window_height)
                                patch=[a,b,a+self.params.dim_window_width,b+self.params.dim_window_height]
                                for box in boxes:
                                    p.append(self.intersection_over_union(box,patch))
                                for v in p:
                                    if v<0.05:
                                        nr+=1
                                c+=1
                    if c==50000:
                        continue
                    patch_img=img[patch[1]:patch[3],patch[0]:patch[2]]
                    negative_examples.append(patch_img)
                    patch_filename = f"{i}{idx}{person}.jpg" 
                    patch_filepath = os.path.join(self.params.dir_neg_examples, patch_filename)
                    cv.imwrite(patch_filepath, patch_img)
        return negative_examples

    
    
    def rotate_image_30_degrees(self,image,angle):
        # Read the image

        # Get image shape
        rows, cols = image.shape

        # Define the rotation angle (30 degrees)
        angle = 30

        # Calculate the rotation matrix
        rotation_matrix = cv.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

        # Apply the rotation to the image
        rotated_image = cv.warpAffine(image, rotation_matrix, (cols, rows))

        return rotated_image
        


            



    def get_positive_descriptors(self):
        # in aceasta functie calculam descriptorii pozitivi
        # vom returna un numpy array de dimensiuni NXD
        # unde N - numar exemplelor pozitive
        # iar D - dimensiunea descriptorului
        # D = (params.dim_window/params.dim_hog_cell - 1) ^ 2 * params.dim_descriptor_cell (fetele sunt patrate)

        images_path = os.path.join(self.params.dir_pos_examples, '*.jpg')
        files = glob.glob(images_path)
        num_images = len(files)
        positive_descriptors = []
        print('Calculam descriptorii pt %d imagini pozitive...' % num_images)
        for i in range(num_images):
            print('Procesam exemplul pozitiv numarul %d...' % i)
            img = cv.imread(files[i], cv.IMREAD_GRAYSCALE)
            # TODO: sterge
            features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                           cells_per_block=(2, 2), feature_vector=True)
            print(len(features))

            positive_descriptors.append(features)
            if self.params.use_flip_images:
                features = hog(np.fliplr(img), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                positive_descriptors.append(features)
                '''
                features = hog(np.fliplr(self.rotate_image_30_degrees(img,30)), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                positive_descriptors.append(features)
                
                features = hog(np.fliplr(self.rotate_image_30_degrees(img,30)), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                positive_descriptors.append(features)
             
            features=hog(self.rotate_image_30_degrees(img,30), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
            positive_descriptors.append(features)
            
            features=hog(self.rotate_image_30_degrees(img,270), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
            '''
            

        positive_descriptors = np.array(positive_descriptors)
        return positive_descriptors

    def get_negative_descriptors(self):
        # in aceasta functie calculam descriptorii negativi
        # vom returna un numpy array de dimensiuni NXD
        # unde N - numar exemplelor negative
        # iar D - dimensiunea descriptorului
        # avem 274 de imagini negative, vream sa avem self.params.number_negative_examples (setat implicit cu 10000)
        # de exemple negative, din fiecare imagine vom genera aleator self.params.number_negative_examples // 274
        # patch-uri de dimensiune 36x36 pe care le vom considera exemple negative

        images_path = os.path.join(self.params.dir_neg_examples, '*.jpg')
        files = glob.glob(images_path)
        num_images = len(files)
        num_negative_per_image = self.params.number_negative_examples // num_images
        negative_descriptors = []
        print('Calculam descriptorii pt %d imagini negative' % num_images)
        for i in range(num_images):
            print('Procesam exemplul negativ numarul %d...' % i)
            img = cv.imread(files[i], cv.IMREAD_GRAYSCALE)
            descr = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                        cells_per_block=(2, 2), feature_vector=False)
            negative_descriptors.append(descr.flatten())
        negative_descriptors = np.array(negative_descriptors)
        return negative_descriptors
    
    
    
    def get_extra_descriptors(self):
    # in aceasta functie calculam descriptorii pozitivi
    # vom returna un numpy array de dimensiuni NXD
    # unde N - numar exemplelor pozitive
    # iar D - dimensiunea descriptorului
    # D = (params.dim_window/params.dim_hog_cell - 1) ^ 2 * params.dim_descriptor_cell (fetele sunt patrate)

        images_path = os.path.join(f"{self.params.base_dir}/extra_test1", '*.jpg')
        files = glob.glob(images_path)
        num_images = len(files)
        extra_descriptors = []
        print('Calculam descriptorii pt %d imagini negative + extra...' % num_images)
        for i in range(num_images):
            print('Procesam exemplul pozitiv numarul %d...' % i)
            img = cv.imread(files[i], cv.IMREAD_GRAYSCALE)
            # TODO: sterge
            features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                           cells_per_block=(2, 2), feature_vector=True)
            print(len(features))

            extra_descriptors.append(features)
        return extra_descriptors

    def train_classifier(self, training_examples, train_labels,person=None):
        svm_file_name = os.path.join(self.params.dir_save_files, 'best_model_%d_%d_%d' %
                                     (self.params.dim_hog_cell, self.params.number_negative_examples,
                                      self.params.number_positive_examples))
        if person:
            svm_file_name+="_"+person
        if os.path.exists(svm_file_name):
            self.best_model = pickle.load(open(svm_file_name, 'rb'))
            return

        best_accuracy = 0
        best_c = 0
        best_model = None
        Cs = [10 ** -5, 10 ** -4,  10 ** -3,  10 ** -2, 10 ** -1,1]
        for c in Cs:
            print('Antrenam un clasificator pentru c=%f' % c)
            model = LinearSVC(C=c)
            model.fit(training_examples, train_labels)
            acc = model.score(training_examples, train_labels)
            print(acc)
            if acc > best_accuracy:
                best_accuracy = acc
                best_c = c
                best_model = deepcopy(model)
                
        
                

        print('Performanta clasificatorului optim pt c = %f' % best_c)
        # salveaza clasificatorul
        pickle.dump(best_model, open(svm_file_name, 'wb'))

        # vizualizeaza cat de bine sunt separate exemplele pozitive de cele negative dupa antrenare
        # ideal ar fi ca exemplele pozitive sa primeasca scoruri > 0, iar exemplele negative sa primeasca scoruri < 0
        scores = best_model.decision_function(training_examples)
        self.best_model = best_model
        positive_scores = scores[train_labels > 0]
        negative_scores = scores[train_labels <= 0]
        


        plt.plot(np.sort(positive_scores))
        plt.plot(np.zeros(len(positive_scores)))
        plt.plot(np.sort(negative_scores))
        plt.xlabel('Nr example antrenare')
        plt.ylabel('Scor clasificator')
        plt.title('Distributia scorurilor clasificatorului pe exemplele de antrenare')
        plt.legend(['Scoruri exemple pozitive', '0', 'Scoruri exemple negative'])
        plt.show()



    def non_maximal_suppression(self, image_detections, image_scores, image_size):
        """
        Detectiile cu scor mare suprima detectiile ce se suprapun cu acestea dar au scor mai mic.
        Detectiile se pot suprapune partial, dar centrul unei detectii nu poate
        fi in interiorul celeilalte detectii.
        :param image_detections:  numpy array de dimensiune NX4, unde N este numarul de detectii.
        :param image_scores: numpy array de dimensiune N
        :param image_size: tuplu, dimensiunea imaginii
        :return: image_detections si image_scores care sunt maximale.
        """

        # xmin, ymin, xmax, ymax
        x_out_of_bounds = np.where(image_detections[:, 2] > image_size[1])[0]
        y_out_of_bounds = np.where(image_detections[:, 3] > image_size[0])[0]
        #print(x_out_of_bounds, y_out_of_bounds)
        image_detections[x_out_of_bounds, 2] = image_size[1]
        image_detections[y_out_of_bounds, 3] = image_size[0]
        sorted_indices = np.flipud(np.argsort(image_scores))
        sorted_image_detections = image_detections[sorted_indices]
        sorted_scores = image_scores[sorted_indices]

        is_maximal = np.ones(len(image_detections)).astype(bool)
        iou_threshold = 0.3
        for i in range(len(sorted_image_detections) - 1):
            if is_maximal[i] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                for j in range(i + 1, len(sorted_image_detections)):
                    if is_maximal[j] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                        if self.intersection_over_union(sorted_image_detections[i],sorted_image_detections[j]) > iou_threshold:is_maximal[j] = False
                        else:  # verificam daca centrul detectiei este in mijlocul detectiei cu scor mai mare
                            c_x = (sorted_image_detections[j][0] + sorted_image_detections[j][2]) / 2
                            c_y = (sorted_image_detections[j][1] + sorted_image_detections[j][3]) / 2
                            if sorted_image_detections[i][0] <= c_x <= sorted_image_detections[i][2] and \
                                    sorted_image_detections[i][1] <= c_y <= sorted_image_detections[i][3]:
                                is_maximal[j] = False
        return sorted_image_detections[is_maximal], sorted_scores[is_maximal]

    def run(self):
        """
        Aceasta functie returneaza toate detectiile ( = ferestre) pentru toate imaginile din self.params.dir_test_examples
        Directorul cu numele self.params.dir_test_examples contine imagini ce
        pot sau nu contine fete. Aceasta functie ar trebui sa detecteze fete atat pe setul de
        date MIT+CMU dar si pentru alte imagini
        Functia 'non_maximal_suppression' suprimeaza detectii care se suprapun (protocolul de evaluare considera o detectie duplicata ca fiind falsa)
        Suprimarea non-maximelor se realizeaza pe pentru fiecare imagine.
        :return:
        detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
        detections[i, :] = [x_min, y_min, x_max, y_max]
        scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
        file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
        (doar numele, nu toata calea).
        """
        scales=[1.         ,0.87055,    0.75785828, 0.66069345, 0.57622779, 0.50270779,
 0.43855988, 0.38239679, 0.33287077, 0.28877435, 0.2499512,  0.21516408,
 0.18401213, 0.15607649, 0.13190697]

        test_images_path = os.path.join(self.params.dir_test_examples, '*.jpg')
        test_files = glob.glob(test_images_path)
        detections = None  # array cu toate detectiile pe care le obtinem
        scores = np.array([])  # array cu toate scorurile pe care le obtinem
        file_names = np.array([])  # array cu fisiele, in aceasta lista fisierele vor aparea de mai multe ori, pentru fiecare
        # detectie din imagine, numele imaginii va aparea in aceasta lista
        w = self.best_model.coef_.T
        bias = self.best_model.intercept_[0]
        num_test_images = len(test_files)
        descriptors_to_return = []
        for i in range(num_test_images):
            start_time = timeit.default_timer()
            print('Procesam imaginea de testare %d/%d..' % (i, num_test_images))
            img = cv.imread(test_files[i], cv.IMREAD_GRAYSCALE)
            img_c=img
            image_scores = np.array([])
            image_detections = np.array([])
            for scale in scales:
                img=self.rescale_image(img_c,scale)
                hog_descriptors = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                      cells_per_block=(2, 2), feature_vector=False)
                num_cols = img.shape[1] // self.params.dim_hog_cell -1
                num_rows = img.shape[0] // self.params.dim_hog_cell -1
                num_cell_in_template_height = self.params.dim_window_height // self.params.dim_hog_cell-1
                num_cell_in_template_width= self.params.dim_window_width // self.params.dim_hog_cell-1 
                

                for y in range(0, num_rows - num_cell_in_template_height):
                    for x in range(0, num_cols - num_cell_in_template_width):
                        descr = hog_descriptors[y:y + num_cell_in_template_height, x:x + num_cell_in_template_width].flatten()
                        score = np.dot(descr, w)[0] + bias
                        if score > self.params.threshold:
                            x_min = int(x * self.params.dim_hog_cell)
                            y_min = int(y * self.params.dim_hog_cell)
                            x_max = int(x * self.params.dim_hog_cell + self.params.dim_window)
                            y_max = int(y * self.params.dim_hog_cell + self.params.dim_window)
                            box=[x_min,y_min,x_max,y_max]
                            new_box=self.rescale_bbox(box,scale,1)
                            if image_detections.size>0:
                                image_detections=np.vstack([image_detections,new_box])
                            else:
                                image_detections=np.array([new_box])
                            image_scores=np.append(image_scores,score)
            if len(image_scores) > 0:
                image_detections, image_scores = self.non_maximal_suppression(np.array(image_detections),
                                                                              np.array(image_scores), img_c.shape)
                
                
            if len(image_scores) > 0:
                if detections is None:
                    detections = image_detections
                else:
                    detections = np.concatenate((detections, image_detections))
                scores = np.append(scores, image_scores)
                short_name = ntpath.basename(test_files[i])
                image_names = [short_name for ww in range(len(image_scores))]
                file_names = np.append(file_names, image_names)

            end_time = timeit.default_timer()
            print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
                  % (i, num_test_images, end_time - start_time))

        return detections, scores, file_names

    def compute_average_precision(self, rec, prec):
        # functie adaptata din 2010 Pascal VOC development kit
        m_rec = np.concatenate(([0], rec, [1]))
        m_pre = np.concatenate(([0], prec, [0]))
        for i in range(len(m_pre) - 1, -1, 1):
            m_pre[i] = max(m_pre[i], m_pre[i + 1])
        m_rec = np.array(m_rec)
        i = np.where(m_rec[1:] != m_rec[:-1])[0] + 1
        average_precision = np.sum((m_rec[i] - m_rec[i - 1]) * m_pre[i])
        return average_precision

    def eval_detections(self,detections, scores, file_names,ground_truth_path):
        ground_truth_file = np.loadtxt(ground_truth_path, dtype='str')
        ground_truth_file_names = np.array(ground_truth_file[:, 0])
        ground_truth_detections = np.array(ground_truth_file[:, 1:], int)

        num_gt_detections = len(ground_truth_detections)  # numar total de adevarat pozitive
        gt_exists_detection = np.zeros(num_gt_detections)
        # sorteazam detectiile dupa scorul lor
        sorted_indices = np.argsort(scores)[::-1]
        file_names = file_names[sorted_indices]
        scores = scores[sorted_indices]
        detections = detections[sorted_indices]

        num_detections = len(detections)
        true_positive = np.zeros(num_detections)
        false_positive = np.zeros(num_detections)
        duplicated_detections = np.zeros(num_detections)

        for detection_idx in range(num_detections):
            indices_detections_on_image = np.where(ground_truth_file_names == file_names[detection_idx])[0]

            gt_detections_on_image = ground_truth_detections[indices_detections_on_image]
            bbox = detections[detection_idx]
            max_overlap = -1
            index_max_overlap_bbox = -1
            for gt_idx, gt_bbox in enumerate(gt_detections_on_image):
                overlap = self.intersection_over_union(bbox, gt_bbox)
                if overlap > max_overlap:
                    max_overlap = overlap
                    index_max_overlap_bbox = indices_detections_on_image[gt_idx]

            # clasifica o detectie ca fiind adevarat pozitiva / fals pozitiva
            if max_overlap >= 0.3:
                if gt_exists_detection[index_max_overlap_bbox] == 0:
                    true_positive[detection_idx] = 1
                    gt_exists_detection[index_max_overlap_bbox] = 1
                else:
                    false_positive[detection_idx] = 1
                    duplicated_detections[detection_idx] = 1
            else:
                false_positive[detection_idx] = 1

        cum_false_positive = np.cumsum(false_positive)
        cum_true_positive = np.cumsum(true_positive)

        rec = cum_true_positive / num_gt_detections
        prec = cum_true_positive / (cum_true_positive + cum_false_positive)
        average_precision = self.compute_average_precision(rec, prec)
        plt.plot(rec, prec, '-')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('All faces: average precision %.3f' % average_precision)
        plt.savefig('precizie_medie_all_faces.png')
        plt.show()
        
    
    def my_check_train(self):
        num_images = 4000
        all_boxes=[]
        negative_examples=[]
        num_negative_per_image = self.params.number_negative_examples // num_images
        people=["fred","barney","wilma","betty"]
        for person in people:
            i=1
            j=0
            folder_path = f"{self.params.base_dir}{person}"
            annotations_path=f"{self.params.base_dir}{person}_annotations.txt"
            with open(annotations_path, 'r') as file:
                lines = file.readlines()
            files = os.listdir(folder_path)
            patches=[]
            for file in files:
                c=0
                file_path = os.path.join(folder_path, file)
                img=cv.imread(file_path)
                boxes=[]
                i_str=str(i)
                while len(i_str)<4:
                    i_str="0"+i_str
                while j<len(lines) and lines[j][0:4]==i_str:
                    box=lines[j].split()[1:5]
                    for m in range(len(box)):
                        box[m]=int(box[m])
                    boxes.append(box)
                    j+=1
                i+=1
                all_boxes.append(boxes)
        return all_boxes
        
    def run_train(self,all_boxes):
        """
        Aceasta functie returneaza toate detectiile ( = ferestre) pentru toate imaginile din self.params.dir_test_examples
        Directorul cu numele self.params.dir_test_examples contine imagini ce
        pot sau nu contine fete. Aceasta functie ar trebui sa detecteze fete atat pe setul de
        date MIT+CMU dar si pentru alte imagini
        Functia 'non_maximal_suppression' suprimeaza detectii care se suprapun (protocolul de evaluare considera o detectie duplicata ca fiind falsa)
        Suprimarea non-maximelor se realizeaza pe pentru fiecare imagine.
        :return:
        detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
        detections[i, :] = [x_min, y_min, x_max, y_max]
        scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
        file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
        (doar numele, nu toata calea).
        """
        scales=[1.         ,0.87055,    0.75785828, 0.66069345, 0.57622779, 0.50270779,
    0.43855988, 0.38239679, 0.33287077, 0.28877435, 0.2499512,  0.21516408]

        #test_images_path = os.path.join(self.params.dir_test_examples, '*.jpg')
        test_images_path=os.path.join(self.params.base_dir,"fred")
        next=["","barney","wilma","betty"]
        j=0
        #test_files = glob.glob(test_images_path)
        detections = None  # array cu toate detectiile pe care le obtinem
        scores = np.array([])  # array cu toate scorurile pe care le obtinem
        file_names = np.array([])  # array cu fisiele, in aceasta lista fisierele vor aparea de mai multe ori, pentru fiecare
        # detectie din imagine, numele imaginii va aparea in aceasta lista
        w = self.best_model.coef_.T
        bias = self.best_model.intercept_[0]
        #num_test_images = len(test_files)
        num_test_images=4000
        descriptors_to_return = []
        test_files = os.listdir(test_images_path)
        new_file=False
        i=0
        while j <4:
            if i==999:
                new_file=True

            start_time = timeit.default_timer()
            print('Procesam imaginea de testare %d/%d..' % (i, num_test_images))
            img = cv.imread(os.path.join(test_images_path,test_files[i]), cv.IMREAD_GRAYSCALE)
            img_c=img
            image_scores = np.array([])
            image_detections = np.array([])
            for scale in scales:
                img=self.rescale_image(img_c,scale)
                hog_descriptors = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                      cells_per_block=(2, 2), feature_vector=False)
                num_cols = img.shape[1] // self.params.dim_hog_cell -1
                num_rows = img.shape[0] // self.params.dim_hog_cell -1
                num_cell_in_template_height = self.params.dim_window_height // self.params.dim_hog_cell-1
                num_cell_in_template_width= self.params.dim_window_width // self.params.dim_hog_cell-1 


                for y in range(0, num_rows - num_cell_in_template_height):
                    for x in range(0, num_cols - num_cell_in_template_width):
                        descr = hog_descriptors[y:y + num_cell_in_template_height, x:x + num_cell_in_template_width].flatten()
                        score = np.dot(descr, w)[0] + bias
                        if score > self.params.threshold:
                            x_min = int(x * self.params.dim_hog_cell)
                            y_min = int(y * self.params.dim_hog_cell)
                            x_max = int(x * self.params.dim_hog_cell + self.params.dim_window)
                            y_max = int(y * self.params.dim_hog_cell + self.params.dim_window)
                            box=[x_min,y_min,x_max,y_max]
                            new_box=self.rescale_bbox(box,scale,1)
                            if image_detections.size>0:
                                image_detections=np.vstack([image_detections,new_box])
                            else:
                                image_detections=np.array([new_box])
                            image_scores=np.append(image_scores,score)
            if len(image_scores) > 0:
                image_detections, image_scores = self.non_maximal_suppression(np.array(image_detections),
                                                                              np.array(image_scores), img_c.shape)
                
            real=[]
            for idx in range(len(image_detections)):
                if image_scores[idx]>3:
                    real.append(image_detections[idx])

            nr=0
            for bbox in real:
                ok=False
                for box in all_boxes[i+j*1000]:
                    if(self.intersection_over_union(bbox,box)>0.15 or is_bbox_inside(bbox,box)):
                        ok=True  
                if ok==False:
                    xmin,ymin,xmax,ymax=bbox
                    image_patch = img_c[(ymin):(ymax), (xmin):(xmax)]
                    image_patch=self.resize_to_specific_size(image_patch,self.params.dim_window_width,self.params.dim_window_height)
                    patch_filename = f"{i+j*1000}_{nr}.jpg" 
                    patch_filepath = os.path.join(self.params.base_dir, f"extra_test1/{patch_filename}")
                    #cv.imshow("",image_patch)
                    #cv.waitKey(0)
                    cv.imwrite(patch_filepath, image_patch)
                nr+=1
                
            if len(image_scores) > 0:
                if detections is None:
                    detections = image_detections
                else:
                    detections = np.concatenate((detections, image_detections))
                scores = np.append(scores, image_scores)
                short_name = ntpath.basename(test_files[i])
                image_names = [short_name for ww in range(len(image_scores))]
                file_names = np.append(file_names, image_names)

            end_time = timeit.default_timer()
            if new_file:
                j+=1
                test_images_path=os.path.join(self.params.base_dir,next[j])
                test_files = os.listdir(test_images_path)
                new_file=False
                i=-1
            i+=1
            print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
                  % (i, num_test_images, end_time - start_time))

        return detections, scores, file_names
    
    

    def get_poz_person_descriptors(self,person):
        positive_descriptors=[]
        path=self.params.dir_pos_examples
        images=os.listdir(path)
        for image in images:
            if image.split(".")[0].split("_")[1]==person:
                full_path=os.path.join(path,image)
                img=cv.imread(full_path,cv.IMREAD_GRAYSCALE)
                features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                positive_descriptors.append(features)
                
                if self.params.use_flip_images:
                    features = hog(np.fliplr(img), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                   cells_per_block=(2, 2), feature_vector=True)
                    positive_descriptors.append(features)
        return np.array(positive_descriptors)
    
    def get_neg_person_descriptors(self,person,negative=[]):
        negative_descriptors=[]
        path=self.params.dir_pos_examples
        images=os.listdir(path)
        i=0
        for image in images:
            if image.split(".")[0].split("_")[1]!=person:
                full_path=os.path.join(path,image)
                img=cv.imread(full_path,cv.IMREAD_GRAYSCALE)
                features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                negative_descriptors.append(features)
                
                if self.params.use_flip_images:
                    features = hog(np.fliplr(img), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                   cells_per_block=(2, 2), feature_vector=True)
                    negative_descriptors.append(features)
        negative_descriptors=np.array(negative_descriptors)
        if len(negative)!=0:
            negative_descriptors = np.concatenate((negative_descriptors, negative), axis=0)
        return (negative_descriptors)
    '''
    def train_recognisition(self):
    
    def face_recognition(self,detections):
    '''
        
        
    def run_person(self,detections,file_names):
        """
        Aceasta functie returneaza toate detectiile ( = ferestre) pentru toate imaginile din self.params.dir_test_examples
        Directorul cu numele self.params.dir_test_examples contine imagini ce
        pot sau nu contine fete. Aceasta functie ar trebui sa detecteze fete atat pe setul de
        date MIT+CMU dar si pentru alte imagini
        Functia 'non_maximal_suppression' suprimeaza detectii care se suprapun (protocolul de evaluare considera o detectie duplicata ca fiind falsa)
        Suprimarea non-maximelor se realizeaza pe pentru fiecare imagine.
        :return:
        detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
        detections[i, :] = [x_min, y_min, x_max, y_max]
        scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
        file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
        (doar numele, nu toata calea).
        """
        scales=[1.         ,0.87055,    0.75785828, 0.66069345, 0.57622779, 0.50270779,
 0.43855988, 0.38239679, 0.33287077, 0.28877435, 0.2499512,  0.21516408,
 0.18401213, 0.15607649, 0.13190697]

        test_images_path = (self.params.dir_test_examples)
        detections_new = None  # array cu toate detectiile pe care le obtinem
        scores = np.array([])  # array cu toate scorurile pe care le obtinem
        file_names_new = np.array([])  # array cu fisiele, in aceasta lista fisierele vor aparea de mai multe ori, pentru fiecare
        # detectie din imagine, numele imaginii va aparea in aceasta lista
        w = self.best_model.coef_.T
        bias = self.best_model.intercept_[0]
        num_test_images = len(detections)
        descriptors_to_return = []
        i=0
        while i < (detections.shape[0]):
            path=os.path.join(test_images_path,file_names[i])
            start_time = timeit.default_timer()
            print('Procesam imaginea de testare %d/%d..' % (i, num_test_images))
            img = cv.imread(path, cv.IMREAD_GRAYSCALE)
            aux=file_names[i]
            while i<detections.shape[0] and file_names[i]==aux:
                image_scores = np.array([])
                image_detections = np.array([])
                x_min, y_min, x_max, y_max=detections[i]
                img_run=img[y_min:y_max,x_min:x_max]
                img_c=img_run
                ok=False
                for scale in scales:
                    if ok:
                        break
                    img_cop=self.rescale_image(img_c,scale)
                    hog_descriptors = hog(img_run, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                          cells_per_block=(2, 2), feature_vector=False)
                    num_cols = img_run.shape[1] // self.params.dim_hog_cell -1
                    num_rows = img_run.shape[0] // self.params.dim_hog_cell -1
                    num_cell_in_template_height = self.params.dim_window_height // self.params.dim_hog_cell-1
                    num_cell_in_template_width= self.params.dim_window_width // self.params.dim_hog_cell-1 
                    for y in range(0, num_rows - num_cell_in_template_height):
                        if ok:
                            break
                        for x in range(0, num_cols - num_cell_in_template_width):
                            descr = hog_descriptors[y:y + num_cell_in_template_height, x:x + num_cell_in_template_width].flatten()
                            score = np.dot(descr, w)[0] + bias
                            if score > self.params.threshold:
                                '''
                                x_min = int(x * self.params.dim_hog_cell)
                                y_min = int(y * self.params.dim_hog_cell)
                                x_max = int(x * self.params.dim_hog_cell + self.params.dim_window)
                                y_max = int(y * self.params.dim_hog_cell + self.params.dim_window)
                                '''
                                new_box=[x_min,y_min,x_max,y_max]
                                #cv.imshow()
                                ok=True
                                if image_detections.size>0:
                                    image_detections=np.vstack([image_detections,new_box])
                                else:
                                    image_detections=np.array([new_box])
                                image_scores=np.append(image_scores,score)
                                break


                if len(image_scores) > 0:
                    if detections_new is None:
                        detections_new = image_detections
                    else:
                        detections_new = np.concatenate((detections_new, image_detections))
                    scores = np.append(scores, image_scores)
                    short_name = ntpath.basename(file_names[i])
                    image_names = [short_name for ww in range(len(image_scores))]
                    file_names_new = np.append(file_names_new, image_names)
                
                i+=1


                end_time = timeit.default_timer()
                print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
                      % (i, num_test_images, end_time - start_time))

        return detections_new, scores, file_names_new
    
    
    


In [ ]:
import cv2 as cv
import os
import numpy as np
import pdb
import ntpath
import glob


def show_detections_without_ground_truth(detections, scores, file_names, params: Parameters):
    """
    Afiseaza si salveaza imaginile adnotate.
    detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
    detections[i, :] = [x_min, y_min, x_max, y_max]
    scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
    file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
    (doar numele, nu toata calea).
    """
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)

    for test_file in test_files:
        image = cv.imread(test_file)
        short_file_name = ntpath.basename(test_file)
        indices_detections_current_image = np.where(file_names == short_file_name)
        current_detections = detections[indices_detections_current_image]
        current_scores = scores[indices_detections_current_image]

        for idx, detection in enumerate(current_detections):
            cv.rectangle(image, (detection[0], detection[1]), (detection[2], detection[3]), (0, 0, 255), thickness=1)
            cv.putText(image, 'score:' + str(current_scores[idx])[:4], (detection[0], detection[1]),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv.imwrite(os.path.join(params.dir_save_files, "detections_" + short_file_name), image)
        print('Apasa orice tasta pentru a continua...')
        cv.imshow('image', np.uint8(image))
        cv.waitKey(0)


def show_detections_with_ground_truth(detections, scores, file_names, params: Parameters):
    """
    Afiseaza si salveaza imaginile adnotate. Deseneaza bounding box-urile prezice si cele corecte.
    detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
    detections[i, :] = [x_min, y_min, x_max, y_max]
    scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
    file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
    (doar numele, nu toata calea).
    """

    ground_truth_bboxes = np.loadtxt(params.path_annotations, dtype='str')
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)

    for test_file in test_files:
        image = cv.imread(test_file)
        short_file_name = ntpath.basename(test_file)
        indices_detections_current_image = np.where(file_names == short_file_name)
        current_detections = detections[indices_detections_current_image]
        current_scores = scores[indices_detections_current_image]

        for idx, detection in enumerate(current_detections):
            cv.rectangle(image, (detection[0], detection[1]), (detection[2], detection[3]), (0, 0, 255), thickness=1)
            cv.putText(image, 'score:' + str(current_scores[idx])[:4], (detection[0], detection[1]),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        annotations = ground_truth_bboxes[ground_truth_bboxes[:, 0] == short_file_name]

        # show ground truth bboxes
        for detection in annotations:
            cv.rectangle(image, (int(detection[1]), int(detection[2])), (int(detection[3]), int(detection[4])), (0, 255, 0), thickness=1)

        cv.imwrite(os.path.join(params.dir_save_files, "detections_" + short_file_name), image)
        print('Apasa orice tasta pentru a continua...')
        cv.imshow('image', np.uint8(image))
        cv.waitKey(0)
        
        
def eval_detections_character(detections, scores, file_names,ground_truth_path,character):
    ground_truth_file = np.loadtxt(ground_truth_path, dtype='str')
    ground_truth_file_names = np.array(ground_truth_file[:, 0])
    ground_truth_detections = np.array(ground_truth_file[:, 1:], int)

    num_gt_detections = len(ground_truth_detections)  # numar total de adevarat pozitive
    gt_exists_detection = np.zeros(num_gt_detections)
    # sorteazam detectiile dupa scorul lor
    sorted_indices = np.argsort(scores)[::-1]
    file_names = file_names[sorted_indices]
    scores = scores[sorted_indices]
    detections = detections[sorted_indices]

    num_detections = len(detections)
    true_positive = np.zeros(num_detections)
    false_positive = np.zeros(num_detections)
    duplicated_detections = np.zeros(num_detections)

    for detection_idx in range(num_detections):
        indices_detections_on_image = np.where(ground_truth_file_names == file_names[detection_idx])[0]

        gt_detections_on_image = ground_truth_detections[indices_detections_on_image]
        bbox = detections[detection_idx]
        max_overlap = -1
        index_max_overlap_bbox = -1
        for gt_idx, gt_bbox in enumerate(gt_detections_on_image):
            overlap = facial_detector.intersection_over_union(bbox, gt_bbox)
            if overlap > max_overlap:
                max_overlap = overlap
                index_max_overlap_bbox = indices_detections_on_image[gt_idx]

        # clasifica o detectie ca fiind adevarat pozitiva / fals pozitiva
        if max_overlap >= 0.3:
            if gt_exists_detection[index_max_overlap_bbox] == 0:
                true_positive[detection_idx] = 1
                gt_exists_detection[index_max_overlap_bbox] = 1
            else:
                false_positive[detection_idx] = 1
                duplicated_detections[detection_idx] = 1
        else:
            false_positive[detection_idx] = 1

    cum_false_positive = np.cumsum(false_positive)
    cum_true_positive = np.cumsum(true_positive)

    rec = cum_true_positive / num_gt_detections
    prec = cum_true_positive / (cum_true_positive + cum_false_positive)
    average_precision = facial_detector.compute_average_precision(rec, prec)
    plt.plot(rec, prec, '-')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(character + ' faces: average precision %.3f' % average_precision)
    plt.savefig('precizie_medie_' + character + '.png')
    plt.show()





In [ ]:


params: Parameters = Parameters()
params.dim_window=36
params.dim_window_width = 36
params.dim_window_height = 36# exemplele pozitive (fete de oameni cropate) au 36x36 pixeli
params.dim_hog_cell = 6  # dimensiunea celulei
params.overlap = 0.3
params.number_positive_examples = 6977  # numarul exemplelor pozitive
params.number_negative_examples = 72283  # numarul exemplelor negative

params.threshold = 0 # toate ferestrele cu scorul > threshold si maxime locale devin detectii
params.has_annotations = True

params.use_hard_mining = False  # (optional)antrenare cu exemple puternic negative
params.use_flip_images = True  # adauga imaginile cu fete oglindite

if params.use_flip_images:
    params.number_positive_examples *= 2

facial_detector: FacialDetector = FacialDetector(params)
#good=facial_detector.get_positive_examples()
#bad=facial_detector.get_negative_examples()
#print(len(good),len(bad))

# Pasii 1+2+3. Incarcam exemplele pozitive (cropate) si exemple negative generate
# verificam daca sunt deja existente
positive_features_path = os.path.join(params.dir_save_files, 'descriptoriExemplePozitive_' + str(params.dim_hog_cell) + '_' +
                        str(params.number_positive_examples) + '.npy')
if os.path.exists(positive_features_path):
    positive_features = np.load(positive_features_path)
    print('Am incarcat descriptorii pentru exemplele pozitive',len(positive_features))
else:
    print('Construim descriptorii pentru exemplele pozitive:')
    positive_features = facial_detector.get_positive_descriptors()
    np.save(positive_features_path, positive_features)
    print('Am salvat descriptorii pentru exemplele pozitive in fisierul %s' % positive_features_path)

# exemple negative
negative_features_path = os.path.join(params.dir_save_files, 'descriptoriExempleNegative_' + str(params.dim_hog_cell) + '_' +
                        str(params.number_negative_examples) + '.npy')
if os.path.exists(negative_features_path):
    negative_features = np.load(negative_features_path)
    print('Am incarcat descriptorii pentru exemplele negative')
else:
    print('Construim descriptorii pentru exemplele negative:')
    negative_features = facial_detector.get_extra_descriptors()
    np.save(negative_features_path, negative_features)
    print('Am salvat descriptorii pentru exemplele negative in fisierul %s' % negative_features_path)

# Pasul 4. Invatam clasificatorul liniar
print("Positive Features Shape:", np.squeeze(positive_features).shape)
print("Negative Features Shape:", np.squeeze(negative_features).shape)

training_examples = np.concatenate((np.squeeze(positive_features), np.squeeze(negative_features)), axis=0)
train_labels = np.concatenate((np.ones(positive_features.shape[0]), np.zeros(np.array(negative_features).shape[0])))
#print(len(np.ones(params.number_positive_examples)), len(np.zeros(negative_features.shape[0])))
facial_detector.train_classifier(training_examples, train_labels)

# Pasul 5. (optional) Antrenare cu exemple puternic negative (detectii cu scor >0 din cele 274 de imagini negative)
# Daca implementati acest pas ar trebui sa modificati functia FacialDetector.run()
# astfel incat sa va returneze descriptorii detectiilor cu scor > 0 din cele 274 imagini negative
# completati codul in continuare
# TODO:  (optional)  completeaza codul in continuare

detections, scores, file_names = facial_detector.run()
np.save(os.path.join(params.sol_path, "task1/detections_all_faces.npy"), detections)
np.save(os.path.join(params.sol_path, "task1/scores_all_faces.npy"), scores)
np.save(os.path.join(params.sol_path, "task1/file_names_all_faces.npy"), file_names)

In [ ]:
def run(person):
    params.dir_pos_examples=os.path.join(params.base_dir, "exemplePozitive")
    if person =="fred":
        params.dim_window_width = 36
        params.dim_window_height = 48
        params.number_positive_examples=2
        params.dim_hog_cell=6
        params.number_negative_examples =  67511
    if person =="barney":
        params.dim_window_width = 36
        params.dim_window_height = 36
        params.number_positive_examples=13954
        params.dim_hog_cell=6
        params.number_negative_examples =  72283
    if person =="wilma":
        params.dim_window_width = 36
        params.dim_window_height = 48
        params.number_positive_examples=1
        params.dim_hog_cell=4
        params.number_negative_examples =  67511
    if person =="betty":
        params.dim_window_width = 36
        params.dim_window_height = 48
        params.number_positive_examples=1
        params.dim_hog_cell=4
        params.number_negative_examples =  67511
        
    params.threshold=0
    positive_features_path = os.path.join(params.dir_save_files, 'descriptoriExemplePozitive_' +person+'_'+ str(params.dim_hog_cell) + '_' +
                            str(params.number_positive_examples) + '.npy')
    if os.path.exists(positive_features_path):
        positive_features_2 = np.load(positive_features_path)
        print('Am incarcat descriptorii pentru exemplele pozitive',len(positive_features))
    else:
        print('Construim descriptorii pentru exemplele pozitive:')
        positive_features_2 = facial_detector.get_poz_person_descriptors(person)
        np.save(positive_features_path, positive_features_2)
        print('Am salvat descriptorii pentru exemplele pozitive in fisierul %s' % positive_features_path)
        
        
    print((np.squeeze(positive_features_2).shape,"aici"))
    # exemple negative
    negative_features_path = os.path.join(params.dir_save_files, 'descriptoriExempleNegative_' +person+'_'+ str(params.dim_hog_cell) + '_' +
                            str(params.number_negative_examples) + '.npy')
    if os.path.exists(negative_features_path):
        negative_features_2 = np.load(negative_features_path)
        print('Am incarcat descriptorii pentru exemplele negative')
    else:
        print('Construim descriptorii pentru exemplele negative:')
        negative_features_2 = facial_detector.get_neg_person_descriptors(person,negative_features)
        np.save(negative_features_path, negative_features_2)
        print('Am salvat descriptorii pentru exemplele negative in fisierul %s' % negative_features_path)
        
    print((np.squeeze(negative_features_2).shape,"aici"))
    
    training_examples = np.concatenate((np.squeeze(positive_features_2), np.squeeze(negative_features_2)), axis=0)
    train_labels = np.concatenate((np.ones(positive_features_2.shape[0]), np.zeros(np.array(negative_features_2).shape[0])))
    facial_detector.train_classifier(training_examples, train_labels,person)
    detections2, scores2, file_names2 = facial_detector.run()
    np.save(os.path.join(params.sol_path, f"task2/detections_{person}.npy"), detections2)
    np.save(os.path.join(params.sol_path, f"task2/scores_{person}.npy"), scores2)
    np.save(os.path.join(params.sol_path, f"task2/file_names_{person}.npy"), file_names2)


In [ ]:
run("fred")
run("barney")
run("betty")
run("wilma")

In [ ]:
def evaluate_results_task1(solution_path,ground_truth_path,verbose = 0):

	#incarca detectiile + scorurile + numele de imagini	
	detections = np.load(solution_path + "detections_all_faces.npy",allow_pickle=True,fix_imports=True,encoding='latin1')
	print(detections.shape)

	scores = np.load(solution_path + "scores_all_faces.npy",allow_pickle=True,fix_imports=True,encoding='latin1')
	print(scores.shape)
	
	file_names = np.load(solution_path + "file_names_all_faces.npy",allow_pickle=True,fix_imports=True,encoding='latin1')
	print(file_names.shape)

	facial_detector.eval_detections(detections, scores, file_names, ground_truth_path)

def evaluate_results_task2(solution_path,ground_truth_path,character, verbose = 0):

	#incarca detectiile + scorurile + numele de imagini	
	detections = np.load(solution_path + "detections_" + character + ".npy",allow_pickle=True,fix_imports=True,encoding='latin1')
	print(detections.shape)

	scores = np.load(solution_path + "scores_"+ character + ".npy",allow_pickle=True,fix_imports=True,encoding='latin1')
	print(scores.shape)
	
	file_names = np.load(solution_path + "file_names_"+ character + ".npy",allow_pickle=True,fix_imports=True,encoding='latin1')
	print(file_names.shape)

	eval_detections_character(detections, scores, file_names, ground_truth_path, character)
  

verbose = 0

#change this on your machine
solution_path_root = params.sol_path
ground_truth_path_root = "validare/"


#task1
solution_path = solution_path_root + "task1/"
ground_truth_path = ground_truth_path_root + "task1_gt_validare.txt"
print(solution_path)
evaluate_results_task1(solution_path, ground_truth_path, verbose)

#pdb.set_trace()

#task2
solution_path = solution_path_root + "task2/"


ground_truth_path = ground_truth_path_root + "task2_fred_gt_validare.txt"
evaluate_results_task2(solution_path, ground_truth_path, "fred", verbose)

ground_truth_path = ground_truth_path_root + "task2_barney_gt_validare.txt"
evaluate_results_task2(solution_path, ground_truth_path, "barney", verbose)

ground_truth_path = ground_truth_path_root + "task2_betty_gt_validare.txt"
evaluate_results_task2(solution_path, ground_truth_path, "betty", verbose)

ground_truth_path = ground_truth_path_root + "task2_wilma_gt_validare.txt"
evaluate_results_task2(solution_path, ground_truth_path, "wilma", verbose)

In [ ]:
def to_yolo_2():
    image_width=480
    image_height=360
    people=["fred","barney","wilma","betty"]
    nr=0
    for person in people:
        i=1
        j=0
        folder_path = f"{params.base_dir}{person}"
        annotations_path=f"{params.base_dir}{person}_annotations.txt"
        with open(annotations_path, 'r') as file:
            lines = file.readlines()
        files = os.listdir(folder_path)
        for file in files:
            text_data=""
            file_path = os.path.join(folder_path, file)
            img=cv.imread(file_path)
            i_str=str(i)
            cv.imwrite(f"C:/Users/rares/darknet-master/build/darknet/x64/data/obj/{nr*1000+i}.jpg",img)
            while len(i_str)<4:
                i_str="0"+i_str
            while j<len(lines) and lines[j][0:4]==i_str:
                poz=lines[j].split()[1:5]
                if lines[j].split()[5]!="unknown":
                    for q in range(len(poz)):
                        poz[q]=int(poz[q])
                    xmin,ymin,xmax,ymax=poz

                    x_center = (xmin + xmax) / 2.0
                    y_center = (ymin + ymax) / 2.0
                    box_width = xmax - xmin
                    box_height = ymax - ymin

                    x_center /= image_width
                    y_center /= image_height
                    box_width /= image_width
                    box_height /= image_height

                    text_data+=str(people.index(lines[j].split()[5]))+" "+str(x_center)+" "+str(y_center)+" "+str(box_width)+" "+str(box_height)+"\n"
                j+=1
                    
                
    

            text_data=text_data[:-1]
            with open(f"C:/Users/rares/darknet-master/build/darknet/x64/data/obj/{nr*1000+i}.txt", 'w') as file:
                file.write(text_data)
            i+=1
        nr+=1
        

def to_yolo_1():
    image_width=480
    image_height=360
    people=["fred","barney","wilma","betty"]
    nr=0
    for person in people:
        i=1
        j=0
        folder_path = f"{params.base_dir}{person}"
        annotations_path=f"{params.base_dir}{person}_annotations.txt"
        with open(annotations_path, 'r') as file:
            lines = file.readlines()
        files = os.listdir(folder_path)
        for file in files:
            text_data=""
            file_path = os.path.join(folder_path, file)
            img=cv.imread(file_path)
            i_str=str(i)
            cv.imwrite(f"C:/Users/rares/darknet-master/build/darknet/x64/data/obj/{nr*1000+i}.jpg",img)
            while len(i_str)<4:
                i_str="0"+i_str
            while j<len(lines) and lines[j][0:4]==i_str:
                poz=lines[j].split()[1:5]
                for q in range(len(poz)):
                    poz[q]=int(poz[q])
                xmin,ymin,xmax,ymax=poz

                x_center = (xmin + xmax) / 2.0
                y_center = (ymin + ymax) / 2.0
                box_width = xmax - xmin
                box_height = ymax - ymin

                x_center /= image_width
                y_center /= image_height
                box_width /= image_width
                box_height /= image_height

                text_data+=str(0)+" "+str(x_center)+" "+str(y_center)+" "+str(box_width)+" "+str(box_height)+"\n"
                j+=1
                    
                

            text_data=text_data[:-1]
            with open(f"C:/Users/rares/darknet-master/build/darknet/x64/data/obj/{nr*1000+i}.txt", 'w') as file:
                file.write(text_data)
            i+=1
        nr+=1
               
def train_path():
    text_data=""
    for i in range(1,4001):
        text_data+=f"data/obj/{i}.jpg"+"\n"
    text_data=text_data[:-1]
    with open(f"C:/Users/rares/darknet-master/build/darknet/x64/data/train.txt", 'w') as file:
            file.write(text_data)
    

In [ ]:
to_yolo_2()

In [ ]:
to_yolo_1()

In [ ]:
train_path()

In [ ]:
def yolo_run_2(person):
    net = cv.dnn.readNet("C:/Users/rares/Downloads/yolov4-flintstone_last_2.weights", "C:/Users/rares/darknet/yolov4-flintstone-2.cfg")
    classes = []
    with open("C:/Users/rares/darknet/data/obj.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getUnconnectedOutLayersNames()

    validation_folder = params.dir_test_examples
    detections = None  # array cu toate detectiile pe care le obtinem
    scores = np.array([])  # array cu toate scorurile pe care le obtinem
    file_names = np.array([])
    
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)
    i=0

    for file in os.listdir(validation_folder):
        start_time = timeit.default_timer()
        image_path = os.path.join(validation_folder, file)
        img = cv.imread(image_path)
        height, width, _ = img.shape
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(layer_names)
        image_scores = np.array([])
        image_detections = np.array([])

        # Process the output
        for out in outs:
            for detection in out:
                result = detection[5:]
                class_id = np.argmax(result)
                confidence = result[class_id]
                if confidence > 0.0 and classes[class_id]==person:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x_min = max(0, int(center_x - w / 2))
                    y_min = max(0, int(center_y - h / 2))
                    x_max = min(width, int(center_x + w / 2))
                    y_max = min(height, int(center_y + h / 2))

                    box=[x_min,y_min,x_max,y_max]
                    #cv.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    #cv.putText(img, classes[class_id], (x_min, y_min - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    #cv.imshow("",img)
                    cv.waitKey(0)
                    if image_detections.size>0:
                        image_detections=np.vstack([image_detections,box])
                    else:
                        image_detections=np.array([box])
                    image_scores=np.append(image_scores,confidence)
        if len(image_scores) > 0:
            image_detections, image_scores = facial_detector.non_maximal_suppression(np.array(image_detections),
                                                                          np.array(image_scores), img.shape)

        
        if len(image_scores) > 0:
            if detections is None:
                detections = image_detections
            else:
                detections = np.concatenate((detections, image_detections))
            scores = np.append(scores, image_scores)
            short_name = ntpath.basename(test_files[i])
            image_names = [short_name for ww in range(len(image_scores))]
            file_names = np.append(file_names, image_names)
        i+=1
        end_time = timeit.default_timer()
        print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
              % (i,200, end_time - start_time))
        
    np.save(os.path.join(params.sol_path, f"bonus_task2/detections_{person}.npy"), detections)
    np.save(os.path.join(params.sol_path, f"bonus_task2/scores_{person}.npy"), scores)
    np.save(os.path.join(params.sol_path, f"bonus_task2/file_names_{person}.npy"), file_names)


    return detections, scores, file_names
                          
ground_truth_path = "validare/" + "task2_betty_gt_validare.txt" 
detections4,scores4,file_names4=yolo_run_2("Betty")





In [ ]:
eval_detections_character(detections4,scores4,file_names4,ground_truth_path,"Betty")
show_detections_without_ground_truth(detections4, scores4, file_names4, params)

In [ ]:
def yolo_run_1():
    net = cv.dnn.readNet("C:/Users/rares/Downloads/yolov4-flintstone-1_1000.weights", "C:/Users/rares/darknet/yolov4-flintstone-1.cfg")
    classes = []
    with open("C:/Users/rares/darknet/data/obj.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getUnconnectedOutLayersNames()

    validation_folder = params.dir_test_examples
    detections = None  # array cu toate detectiile pe care le obtinem
    scores = np.array([])  # array cu toate scorurile pe care le obtinem
    file_names = np.array([])
    
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)
    i=0

    for file in os.listdir(validation_folder):
        start_time = timeit.default_timer()
        image_path = os.path.join(validation_folder, file)
        img = cv.imread(image_path)
        height, width, _ = img.shape
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(layer_names)
        image_scores = np.array([])
        image_detections = np.array([])

        # Process the output
        for out in outs:
            for detection in out:
                result = detection[5:]
                class_id = np.argmax(result)
                confidence = result[class_id]
                if confidence > 0.0:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x_min = max(0, int(center_x - w / 2))
                    y_min = max(0, int(center_y - h / 2))
                    x_max = min(width, int(center_x + w / 2))
                    y_max = min(height, int(center_y + h / 2))

                    box=[x_min,y_min,x_max,y_max]
                    #cv.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    #cv.putText(img, classes[class_id], (x_min, y_min - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    if image_detections.size>0:
                        image_detections=np.vstack([image_detections,box])
                    else:
                        image_detections=np.array([box])
                    image_scores=np.append(image_scores,confidence)
        if len(image_scores) > 0:
            image_detections, image_scores = facial_detector.non_maximal_suppression(np.array(image_detections),
                                                                          np.array(image_scores), img.shape)

        
        if len(image_scores) > 0:
            if detections is None:
                detections = image_detections
            else:
                detections = np.concatenate((detections, image_detections))
            scores = np.append(scores, image_scores)
            short_name = ntpath.basename(test_files[i])
            image_names = [short_name for ww in range(len(image_scores))]
            file_names = np.append(file_names, image_names)
        i+=1
        end_time = timeit.default_timer()
        print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
              % (i,200, end_time - start_time))


    np.save(os.path.join(params.sol_path, f"bonus_task1/detections.npy"), detections)
    np.save(os.path.join(params.sol_path, f"task2/scores.npy"), scores)
    np.save(os.path.join(params.sol_path, f"task2/file_names.npy"), file_names)
    return detections, scores, file_names
                          
ground_truth_path = ground_truth_path_root + "task1_gt_validare.txt"             
detections4, scores4, file_names4 = yolo_run_1()
facial_detector.eval_detections(detections4,scores4,file_names4,ground_truth_path)

